# DataInsights #1/session2

- make a clean data set for volunteers to look at in second session

This cleans the WALDEC data set from: https://www.data.gouv.fr/fr/datasets/repertoire-national-des-associations/

#### how many lines does original csv have?

I wonder how many we are missing?

In [1]:
path = './../../data/rna_waldec_20191001/rna_waldec_20191001.csv'

import csv

with open(path) as csvfile:
    reader = csv.DictReader(csvfile)
    
    len(list(reader))

#### Load data with pandas

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv(path, sep=';', error_bad_lines=False, engine='python')

In [4]:
df.shape

(1733433, 39)

In [5]:
df.head(3)

,id,id_ex,siret,rup_mi,gestion,date_creat,date_decla,date_publi,date_disso,nature,...,adrg_distrib,adrg_codepostal,adrg_achemine,adrg_pays,dir_civilite,siteweb,publiweb,observation,position,maj_time
0,W632007046,NaN,NaN,NaN,632P,2014-06-17,2015-01-21,2014-06-28,2015-01-20,D,...,NaN,63000,CLERMONT-FERRAND,FRANCE,PP,NaN,0.0,NaN,D,2015-02-09 12:51:34
1,W012000002,2/08825,NaN,NaN,691P,1983-10-10,2004-03-11,0001-01-01,0001-01-01,D,...,NaN,69100,villeurbanne,FRANCE,PM,NaN,1.0,NaN,A,2008-02-12 14:40:18
2,W012000005,0012011852,NaN,NaN,691P,2003-09-01,2005-01-25,0001-01-01,0001-01-01,D,...,NaN,69270,FONTAINE SUR SAONE,FRANCE,PM,NaN,1.0,NaN,A,2007-04-05 09:41:02


#### find proportion missing from each column

In [6]:
pd.DataFrame(df.isnull().sum()/df.shape[0]).sort_values(by=0, ascending=False).head(20)

,0
adrs_repetition,0.998324
rup_mi,0.996639
siret,0.981590
adrg_declarant,0.976880
siteweb,0.968481
adrg_complemgeo,0.948997
adrg_distrib,0.906085
observation,0.783862
adrg_complemid,0.733536
adrs_complement,0.662397


#### dropcolumns that are not of use

There are many missing values in many columns:

In [7]:
cols_lots_missing_values = [
#     'adrs_repetition',
    'rup_mi',
    'siret',
#     'adrg_declarant',
    'siteweb',
#     'adrg_complemgeo',
#     'adrg_distrib',
]

Some columns look non-useful:

In [8]:
cols_notinteresting = [
    'observation', # this seems to be notes about the admin on the database
#     'id_ex', # this is an old ref, 50% are missing, there is a newer one 'id'
    'date_decla', # we have date created which will do, same for col below
    'date_publi',
    'gestion', #where the associations were recored, presumably online/prefecture.
    'publiweb', # can publish on web or not
    'maj_time', # time aded to database
]


In [9]:
cols_todrop = cols_lots_missing_values + cols_notinteresting
dff = df.drop(cols_todrop,axis=1)

In [10]:
dff.shape

(1733433, 30)

In [11]:
cols_todrop

['rup_mi',
 'siret',
 'siteweb',
 'observation',
 'date_decla',
 'date_publi',
 'gestion',
 'publiweb',
 'maj_time']

In [12]:
for  i in dff.columns:
    print(i)

id
id_ex
date_creat
date_disso
nature
groupement
titre
titre_court
objet
objet_social1
objet_social2
adrs_complement
adrs_numvoie
adrs_repetition
adrs_typevoie
adrs_libvoie
adrs_distrib
adrs_codeinsee
adrs_codepostal
adrs_libcommune
adrg_declarant
adrg_complemid
adrg_complemgeo
adrg_libvoie
adrg_distrib
adrg_codepostal
adrg_achemine
adrg_pays
dir_civilite
position


In [13]:
cols_to_keep = ['id',
                'id_ex',
'date_creat',
'date_disso',
'nature',
'groupement',
'objet_social1',
'adrs_codepostal',
]

In [14]:
dff = df[cols_to_keep]

#### merge table of objet_social1, and objet_social2 so have text description

In [15]:
objet_desc = pd.read_excel('./../../data/Fichier_RNA_-_Nomenclature_Complete_Objet_Social_.xlsx')

There are some address values in the objet columns in the waldoc data set. Clean those and replace with Nans now.

In [16]:
poss_values = objet_desc['Code Objet Social'].values

def filter_func(x):
    ""
    value = np.nan
    if len(str(x)) == 6:
        value=x
    return(value)

# filter_func(11111)

dff.objet_social1 = dff.objet_social1.apply(filter_func)
# dff.objet_social2 = dff.objet_social2.apply(filter_func)

C:\Anaconda3\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [17]:
objet_desc['Code Objet Social'].dtype

dtype('int64')

In [18]:
dff.objet_social1.isnull().sum()

113

There are some missing values in objet_social1 & 2. Dropping these rows as wont merge otherwise. We lose about 300 associaitons.

In [19]:
dff.shape

(1733433, 8)

In [20]:
dff.dropna(subset=['objet_social1'],inplace=True)

dff.shape

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


(1733320, 8)

erroneous_values = ['40 000','023àà(','011.75']

dff.drop(index = dff[dff.objet_social2.isin( erroneous_values)].index, inplace=True)

In [21]:
dff.shape

(1733320, 8)

In [22]:
dff.objet_social1 = dff.objet_social1.astype(int)
# dff.objet_social2 = dff.objet_social2.astype(int)

In [23]:
dff = dff.merge(objet_desc, how='left', left_on='objet_social1', right_on='Code Objet Social')
dff.drop(columns=['Code Objet Social'],inplace=True)
dff.rename({'Objet Social':'objet_social1_desc'},axis=1, inplace=True)
dff.shape

(1733320, 9)

dff = dff.merge(objet_desc, how='left', left_on='objet_social2', right_on='Code Objet Social')
dff.drop(columns=['Code Objet Social'],inplace=True)
dff.rename({'Objet Social':'objet_social2_desc'}, axis=1, inplace=True)
dff.shape

#### create flag for association no longer active 


dff['association_active'] = 0
dff.loc[dff.position == 'A', 'association_active'] = 1

dff.association_active.value_counts()

#### check through columns and check how many lines are messed up with wrong types of data

If there are not too many then we just delete them i think.
- code_postal, len 5
- objet_social1, len 6

#### clean adress fields

## export data to csv

- put columns in nice order

dff.rename({'Objet Social_x':'objet_social2_desc'},axis=1, inplace=True)

In [24]:
dff.head()

,id,id_ex,date_creat,date_disso,nature,groupement,objet_social1,adrs_codepostal,objet_social1_desc
0,W632007046,NaN,2014-06-17,2015-01-20,D,S,6030,63000.0,"chant choral, musique"
1,W012000002,2/08825,1983-10-10,0001-01-01,D,S,6070,69100.0,"théâtre, marionnettes, cirque, spectacles de v..."
2,W012000005,0012011852,2003-09-01,0001-01-01,D,S,20000,69270.0,"associations caritatives, humanitaires, aide a..."
3,W012000006,0012010354,1999-02-11,2008-02-27,D,S,6000,69008.0,"culture, pratiques d\’activités artistiques, c..."
4,W033002617,NaN,2008-05-19,0001-01-01,D,S,11075,3700.0,"Football (football, futsal)"


In [25]:
dff.columns

Index(['id', 'id_ex', 'date_creat', 'date_disso', 'nature', 'groupement',
       'objet_social1', 'adrs_codepostal', 'objet_social1_desc'],
      dtype='object')

# import non-waldec dataset

In [26]:
df_imp = pd.read_csv('./../../data/rna_import_20191001/rna_import_20191001.csv', sep=';',engine='python')

In [27]:
cols_to_keep

['id',
 'id_ex',
 'date_creat',
 'date_disso',
 'nature',
 'groupement',
 'objet_social1',
 'adrs_codepostal']

In [28]:
df_imp.head()

,id,id_ex,siret,gestion,date_creat,date_publi,nature,groupement,titre,objet,...,adr3,adrs_codepostal,libcom,adrs_codeinsee,dir_civilite,siteweb,observation,position,rup_mi,maj_time
0,751P00085568,00085568,NaN,751P,1988-04-15,0001-01-01,D,S,INDIGO,dvpt et démocratisation de la musique,...,NaN,75014,PARIS,75114,PM,NaN,W751085568,R,NaN,2013-07-10 07:33:24
1,751P00184848,00184848,NaN,751P,2008-01-18,2008-02-23,D,S,COMITE DE GESTION RUE PINEL,la gestion des locaux et des services nécessai...,...,NaN,75013,PARIS,75113,PM,NaN,W751184848,R,NaN,2013-07-16 07:30:17
2,751P00188854,00188854,NaN,751P,2008-11-10,0001-01-01,D,S,HELP MEDIA,proposer aux personnes actives et non actives ...,...,NaN,75006,PARIS,75106,PM,NaN,W751188854,R,NaN,2013-09-23 09:55:18
3,751P00100798,00100798,NaN,751P,1991-07-25,0001-01-01,D,S,CENTRE D'ETUDES DE DOCUMENTATION ET D'OBSERVAT...,l'association conduit suscite aide et encourag...,...,NaN,75012,PARIS,75112,PM,NaN,W751100798,R,NaN,2013-09-25 09:27:16
4,061S0061013683,0061013683,NaN,061S,0001-01-01,1997-11-15,D,S,CONTACTS FRANCE UKRAINE,Faire connaitre a tous public l'interet de dev...,...,NaN,06150,CANNES,06029,PM,NaN,W061004727,R,NaN,2013-12-02 12:02:35


- do these associaitons exist in both data sets?

In [53]:
df_imp.tail()

,id,id_ex,siret,gestion,date_creat,date_publi,nature,groupement,titre,objet,...,adr3,adrs_codepostal,libcom,adrs_codeinsee,dir_civilite,siteweb,observation,position,rup_mi,maj_time
1213324,335S0113100,0113100,NaN,335S,1958-03-19,1958-03-27,D,S,ASSOCIATION COMMUNALE DE CHASSE AGREEE DE GOURS,NaN,...,NaN,33660,gours,33191,PM,NaN,W335007024,R,NaN,2019-06-27 06:21:25
1213325,052P5842,5842,NaN,052P,1996-07-10,1996-07-31,D,S,MANIFESTATIONS CULTURELLES PRINTEMPS DU LIVRE,"FAVORISER LES RENCONTRES, LES ECHANGES DANS UN...",...,NaN,05400,VEYNES,00000,PM,NaN,W052007544,R,NaN,2019-06-27 06:58:33
1213326,751P00127896,00127896,NaN,751P,1996-11-20,1996-12-11,D,S,ASSOCIATION DE LOCATAIRES SAINT FARGEAU PLI SL...,organiser la défense des droits des aspiration...,...,NaN,75020,PARIS,75120,PM,NaN,W751127896,R,NaN,2019-06-27 06:21:19
1213327,9G2S9712002529,9712002529,NaN,9G2S,0001-01-01,0001-01-01,D,S,ASSOCIATION ACTION POUR LE PROGRES ET L'AVENIR...,Defendre les interets de la commune de petit c...,...,NaN,97131,PETIT CANAL,97119,PM,NaN,W9G2008797,R,NaN,2019-06-27 06:58:44
1213328,423P0423000058,0423000058,NaN,423P,0001-01-01,1992-07-01,D,S,ASSOCIATION DES LOCATAIRES DES IMMEUBLES RUES ...,Representer les locataires aupres de la societ...,...,NaN,42100,SAINT-ETIENNE,42218,PM,NaN,W423013302,R,NaN,2019-06-28 06:28:43


In [54]:
dff.id_ex

AttributeError: 'DataFrame' object has no attribute 'id_ex'

In [52]:
for i in df_imp.columns:
    print(i)

id
id_ex
siret
gestion
date_creat
date_publi
nature
groupement
titre
objet
objet_social1
objet_social2
adr1
adr2
adr3
adrs_codepostal
libcom
adrs_codeinsee
dir_civilite
siteweb
observation
position
rup_mi
maj_time


In [29]:
dff.to_csv('./../../data_processed/clean_data.csv', index=False)